## Inverse Cooking: Recipe Generation from Food Images

In [2]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import os
from args import get_parser
import pickle
from model import get_model
from torchvision import transforms
from utils.output_utils import prepare_output
from PIL import Image
import time

In [4]:
dataset = pickle.load(open('../data/recipe1m_test.pkl', 'rb'))

In [5]:
dataset

[{'id': '00003a70b1',
  'instructions': ['preheat oven to 350 degrees fahrenheit.',
   'spray pan with non stick cooking spray.',
   'heat milk, water and butter to boiling; stir in contents of both pouches of potatoes; let stand one minute.',
   'stir in corn.',
   'spoon half the potato mixture in pan.',
   'sprinkle half each of cheese and onions; top with remaining potatoes.',
   'sprinkle with remaining cheese and onions.',
   'bake 10 to 15 minutes until cheese is melted.',
   'enjoy !'],
  'tokenized': [['preheat', 'oven', 'to', '350', 'degrees', 'fahrenheit', '.'],
   ['spray', 'pan', 'with', 'non', 'stick', 'cooking', 'spray', '.'],
   ['heat',
    'milk',
    ',',
    'water',
    'and',
    'butter',
    'to',
    'boiling',
    ';',
    'stir',
    'in',
    'contents',
    'of',
    'both',
    'pouches',
    'of',
    'potatoes',
    ';',
    'let',
    'stand',
    'one',
    'minute',
    '.'],
   ['stir', 'in', 'corn', '.'],
   ['spoon', 'half', 'the', 'potato', 'mixtu

Set ```data_dir``` to the path including vocabularies and model checkpoint

In [3]:
data_dir = ''

In [4]:
# code will run in gpu if available and if the flag is set to True, else it will run on cpu
use_gpu = True
device = torch.device('cuda' if torch.cuda.is_available() and use_gpu else 'cpu')
map_loc = None if torch.cuda.is_available() and use_gpu else 'cpu'


# device = torch.device('cpu')
# map_loc = None


In [5]:
print(device)

cuda


In [6]:
# code below was used to save vocab files so that they can be loaded without Vocabulary class
#ingrs_vocab = pickle.load(open(os.path.join(data_dir, 'final_recipe1m_vocab_ingrs.pkl'), 'rb'))
#ingrs_vocab = [min(w, key=len) if not isinstance(w, str) else w for w in ingrs_vocab.idx2word.values()]
#vocab = pickle.load(open(os.path.join(data_dir, 'final_recipe1m_vocab_toks.pkl'), 'rb')).idx2word
#pickle.dump(ingrs_vocab, open('../demo/ingr_vocab.pkl', 'wb'))
#pickle.dump(vocab, open('../demo/instr_vocab.pkl', 'wb'))

ingrs_vocab = pickle.load(open(os.path.join(data_dir, 'ingr_vocab.pkl'), 'rb'))
vocab = pickle.load(open(os.path.join(data_dir, 'instr_vocab.pkl'), 'rb'))

ingr_vocab_size = len(ingrs_vocab)
instrs_vocab_size = len(vocab)
output_dim = instrs_vocab_size

In [7]:
print(ingrs_vocab)

['<end>', 'macaroni', 'cheese', 'celery', 'pimentos', 'mayonnaise', 'vinegar', 'salt', 'dill', 'onion', 'pepper', 'cucumber', 'oil', 'basil', 'gelatin', 'water', 'cool_whip', 'watermelon', 'cracker', 'coconut', 'beef', 'garlic', 'juice', 'soy_sauce', 'cornstarch', 'pineapple', 'nuts', 'chicken', 'tea', 'rhubarb', 'sugar', 'strawberry', 'cake', 'butter', 'flour', 'cinnamon', 'baking_soda', 'baking_powder', 'vanilla', 'zucchini', 'walnuts', 'ginger', 'rice', 'greens', 'fat', 'banana', 'kiwi', 'yogurt', 'seeds', 'tenderloin', 'fennel', 'wine', 'broth', 'brandy', 'liqueur', 'orange', 'lemon', 'sprite', 'ice', 'egg', 'pecans', 'medium_cheddar', 'hamburger', 'oats', 'chili', 'milk', 'corn', 'broccoli', 'oregano', 'extract', 'teriyaki_sauce', 'cream', 'cookie', 'stevia', 'wafers', 'lemonade', 'chips', 'bread', 'half_-_and_-_half', 'syrup', 'crabmeat', 'bacon', 'cilantro', 'seasoning', 'sausage', 'spaghetti', 'pizza_dough', 'tequila', 'triple_sec', 'buttermilk', 'blueberries', 'clove', 'thyme'

In [8]:
print (instrs_vocab_size, ingr_vocab_size)

23231 1488


In [17]:
t = time.time()
import sys; sys.argv=['']; del sys
args = get_parser()
args.maxseqlen = 15
args.ingrs_only=False

# Modify here
args.use_true_ingrs = True
##########################

model = get_model(args, ingr_vocab_size, instrs_vocab_size)
# Load the trained model parameters
model_path = os.path.join('/home/ct2020dl5787/inversecooking/model/checkpoints', 'modelbest.ckpt')
model.load_state_dict(torch.load(model_path, map_location=map_loc))
model.to(device)
model.eval()
model.ingrs_only = False
model.recipe_only = False
model.reduction = 'none'
print ('loaded model')
print ("Elapsed time:", time.time() -t)


loaded model
Elapsed time: 2.7065370082855225


In [18]:
model.recipe_decoder

DecoderTransformer(
  (embed_tokens): Embedding(23231, 512, padding_idx=23230)
  (embed_positions): LearnedPositionalEmbedding(1024, 512, padding_idx=0)
  (layers): ModuleList(
    (0): TransformerDecoderLayer(
      (self_attn): MultiheadAttention(
        (out_proj): Linear(in_features=512, out_features=512, bias=True)
      )
      (cond_att): MultiheadAttention(
        (out_proj): Linear(in_features=512, out_features=512, bias=True)
      )
      (fc1): Linear(in_features=512, out_features=512, bias=True)
      (fc2): Linear(in_features=512, out_features=512, bias=True)
      (layer_norms): ModuleList(
        (0): LayerNorm(torch.Size([512]), eps=1e-05, elementwise_affine=True)
        (1): LayerNorm(torch.Size([512]), eps=1e-05, elementwise_affine=True)
        (2): LayerNorm(torch.Size([512]), eps=1e-05, elementwise_affine=True)
      )
    )
    (1): TransformerDecoderLayer(
      (self_attn): MultiheadAttention(
        (out_proj): Linear(in_features=512, out_features=512, bi

In [19]:
transf_list_batch = []
transf_list_batch.append(transforms.ToTensor())
transf_list_batch.append(transforms.Normalize((0.485, 0.456, 0.406), 
                                              (0.229, 0.224, 0.225)))
to_input_transf = transforms.Compose(transf_list_batch)

In [20]:
greedy = [True, False, False, False]
beam = [-1, -1, -1, -1]
temperature = 1.0
numgens = len(greedy)

Set ```use_urls = True``` to get recipes for images in ```demo_urls```. 

You can also set ```use_urls = False``` and get recipes for images in the path in ```data_dir/test_imgs```.

In [21]:
import requests
from io import BytesIO
import random
from collections import Counter
unpickled_df = pd.read_pickle("df_recipe1023.pkl")
use_urls = True # set to true to load images from demo_urls instead of those in test_imgs folder
show_anyways = False #if True, it will show the recipe even if it's not valid
image_folder = os.path.join('/home/ct2020dl5787/inversecooking/data/'+'demo_imgs')
#TODO1:load data
# if not use_urls:
#     demo_imgs = os.listdir(image_folder)
#     random.shuffle(demo_imgs)

#demo_urls = ['https://food.fnr.sndimg.com/content/dam/images/food/fullset/2013/12/9/0/FNK_Cheesecake_s4x3.jpg.rend.hgtvcom.826.620.suffix/1387411272847.jpeg',
#          'https://www.196flavors.com/wp-content/uploads/2014/10/california-roll-3-FP.jpg']
demo_urls = unpickled_df["url"].tolist()
demo_files = demo_urls if use_urls else demo_imgs

demo_files = demo_files

In [22]:
len(demo_files[:1])

1

In [34]:
info_all = []
err_pic = []
start = time.time()
for img_file in demo_files[1:2]:
    
#    if use_urls:
    response = requests.get(img_file)
    try:
        image = Image.open(BytesIO(response.content))
    
            
#     else:
#         image_path = os.path.join(image_folder, img_file)
#         image = Image.open(image_path).convert('RGB')

        transf_list = []
        transf_list.append(transforms.Resize(256))
        transf_list.append(transforms.CenterCrop(224))
        transform = transforms.Compose(transf_list)

        image_transf = transform(image)
        image_tensor = to_input_transf(image_transf).unsqueeze(0).to(device)

       # plt.imshow(image_transf)
       # plt.axis('off')
      #  plt.show()
      #  plt.close()

        num_valid = 1

        generate_ing = []
        generate_recipt = []
        generate_title = []
        generate_score = []
        for i in range(numgens):
            with torch.no_grad():
                outputs = model.sample(image_tensor, greedy=greedy[i],temperature=temperature, beam=beam[i], true_ingrs=None)
            ingr_ids = outputs['ingr_ids'].cpu().numpy()
            recipe_ids = outputs['recipe_ids'].cpu().numpy()

            outs, valid = prepare_output(recipe_ids[0], ingr_ids[0], ingrs_vocab, vocab)
            #TODO2: Save output  
            generate_ing.append(outs['ingrs'])
            generate_recipt.append(outs['recipe'])
            generate_title.append(outs['title'])
            generate_score.append([valid['is_valid'],valid['score']])

#             if valid['is_valid'] or show_anyways:

#                 print ('RECIPE', num_valid)
#                 num_valid+=1
#                 #print ("greedy:", greedy[i], "beam:", beam[i])

#                 BOLD = '\033[1m'
#                 END = '\033[0m'
#                 print (BOLD + '\nTitle:' + END,outs['title'])

#                 print (BOLD + '\nIngredients:'+ END)
#                 print (', '.join(outs['ingrs']))

#                 print (BOLD + '\nInstructions:'+END)
#                 print ('-'+'\n-'.join(outs['recipe']))

#                 print ('='*20)

#             else:
#                 pass
#                 print ("Not a valid recipe!")
#                 print ("Reason: ", valid['reason'])
        info_all.append([img_file,generate_ing, generate_recipt,generate_title,generate_score])
    except:
        pass
        
print(start-time.time())
    

-13.319494724273682


In [33]:
info_all

[]

In [11]:
generate_data = pd.DataFrame(info_all,columns = ['url','generate_ingre','generate_reci','generate_title','generate_score'])
all_info = pd.merge(unpickled_df, generate_data , how='inner', on=['url'])
all_info.to_pickle('df_recipe1023_all.pkl')

In [ ]:
#TODO3: Evaluation

In [15]:
generate_data.to_pickle('output/generate_1000_backup.pkl')

In [27]:
unpickled_df

,id,ingredients,instructions,url
0,00003a70b1,"[2 12 cups milk, 1 12 cups water, 14 cup butte...","[Preheat oven to 350 degrees Fahrenheit., Spra...","http://img.sndimg.com/food/image/upload/w_512,..."
1,000075604a,"[2 Chicken thighs, 2 tsp Kombu tea, 1 White pe...",[Pierce the skin of the chicken with a fork or...,https://img-global.cpcdn.com/001_recipes/58069...
2,00007bfd16,"[6 -8 cups fresh rhubarb, or, 6 -8 cups frozen...",[Put ingredients in a buttered 9 x 12 x 2-inch...,"http://img.sndimg.com/food/image/upload/w_512,..."
3,000095fc1d,"[8 ounces, weight Light Fat Free Vanilla Yogur...",[Layer all ingredients in a serving dish.],http://tastykitchen.com/recipes/wp-content/upl...
4,0000b1e2b5,"[1 teaspoon fennel seeds, 1 pound pork tenderl...","[Preheat oven to 350F with rack in middle., Cr...",http://assets.epicurious.com/photos/5609a4d662...
...,...,...,...,...
1018,00a2b3f1e4,"[4 whole Fresh Alaskan Halibut Fillets, Skin R...","[Salt and pepper the halibut steaks., Sprinkle...",http://tastykitchen.com/recipes/wp-content/upl...
1019,00a2dbf7a0,"[4 c. low-sodium chicken broth, 2 c. arborio r...",[Heat oven to 400 degrees and arrange a rack i...,http://del.h-cdn.co/assets/cm/15/10/54f6786832...
1020,00a31da144,"[1/2 cup sugar, Juice of 1 orange (or 1/4 cup ...","[1., In a deep saucepan, bring the sugar, oran...",http://assets.epicurious.com/photos/560d99d97b...
1021,00a3285c25,"[7 roma tomatoes, 1 tbsp chicken flavor bouill...","[put tomatoes in pot of water, boil until toma...",https://img-global.cpcdn.com/001_photo_reports...
